In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import time
import gc

# from sklearn.linear_model import RidgeCV, Ridge
# from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split

from lightgbm import LGBMRegressor
# from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
from scipy.stats.stats import pearsonr
from tqdm import tqdm

import gresearch_crypto

# Warningの無効化
import warnings
warnings.simplefilter("ignore")

# データフレームcolumの全表示
pd.set_option("display.max_columns", None)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Load Data

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        # else:
            # df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
df_asset_details = pd.read_csv(r"/kaggle/input/g-research-crypto-forecasting/asset_details.csv").sort_values("Asset_ID")
df_asset_details

In [ ]:
# val = df_asset_details.loc[ df_asset_details["Asset_ID"] == 3, "Asset_Name"]
# val.values[0]

In [ ]:
def read_csv_strict(file_name=r"/kaggle/input/g-research-crypto-forecasting/train.csv"):
    df = pd.read_csv(file_name).pipe(reduce_mem_usage)
    df["datetime"] = pd.to_datetime(df["timestamp"], unit="s")
#     df = df[df["datetime"] > "2019-06-13 00:00:00"]
    df = df[df["datetime"] >= "2020-05-13 00:00:00"]
    return df

In [ ]:
df_train = read_csv_strict()
# df_train.head()
df_supp_train = read_csv_strict(file_name=r"/kaggle/input/g-research-crypto-forecasting/supplemental_train.csv")
# df_supp_train.head()
df_test = read_csv_strict(file_name=r"/kaggle/input/g-research-crypto-forecasting/example_test.csv")
# df_test.head()
df_train = pd.concat([df_train, df_supp_train, df_test])
df_train.head()

In [ ]:
df_train.shape, df_supp_train.shape, df_test.shape

In [ ]:
del df_supp_train, df_test
gc.collect()

In [ ]:
df_train.memory_usage().sum() / 1024**3

## Outliers

In [ ]:
## Negative volume implies lack of liquidity in the market
# df_train[df_train.Volume < 0]

Negative volume indicates that the market lacked liquidity for these instances; in other words, the bid and ask price  failed to overlap.

## Check for NaNs

In [ ]:
## https://www.kaggle.com/c/g-research-crypto-forecasting/discussion/286778
## https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition?scriptVersionId=84976518&cellId=51

# df_train[(df_train.Target.isna()) & (df_train.Asset_ID==8)]
# df_train[df_train.Target.isna()].groupby("Asset_ID")["timestamp"].count()

In [ ]:
# df_train[(df_train.Target.isna()) & (df_train.Asset_ID==8)].describe(percentiles=[0.25, 0.50, 0.75, 0.90, 0.95, 0.99])

## data processing

In [ ]:
df_train = df_train.sort_values(by=["Asset_ID", "timestamp"]
                               ,ascending=True)

df_train.head(20)

In [ ]:
df_train_00=df_train[df_train.Asset_ID==5].copy()
print(len(df_train_00))
df_train_00.drop_duplicates('timestamp',inplace=True)
print(len(df_train_00))
new_index = pd.Index(np.arange(df_train_00.iloc[0]['timestamp'], df_train_00.iloc[-1]['timestamp'] , 60), name="timestamp")
df_train_00=df_train_00.set_index("timestamp").reindex(new_index).reset_index()
df_train_00["trade_count"]=0
col_name=["Open","High","Low","Close","Volume","VWAP","trade_count","Count"]
for name in col_name:
    df_train_00[name].fillna(0, inplace=True)
df_train_00=df_train_00[["Target","Open","High","Low","Close","Volume","VWAP","trade_count","Count","timestamp"]]

In [ ]:
# rename dataframe
df_train_00.columns=["y","AlphaCandleWriter_00_0","AlphaCandleWriter_00_1","AlphaCandleWriter_00_2","AlphaCandleWriter_00_3","AlphaCandleWriter_00_4"
                     ,"AlphaCandleWriter_00_5","AlphaCandleWriter_00_6","AlphaCandleWriter_00_7","AlphaCandleWriter_00_8"]
df_train.head(20)

## Feature

In [ ]:
 conda install -c conda-forge ta-lib

In [ ]:
import threading
import numpy as np
import talib
class get_indicator():
    def __init__(self, lib) -> None:
        self.technical_index = []
        self.technical_header = []
        self.total_cnt_indicator = 0
        self.lib = lib #希望call出的技術指標
        self.open = []
        self.high = [] 
        self.low = []
        self.close = []
        self.volume = []

    #週期指標
    def HT_DCPERIOD(self):
        indicator = talib.HT_DCPERIOD(self.close)
        indicator = np.reshape(indicator, (-1,1))##要先reshape，不然無法append
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("HT_DCPERIOD")
        self.total_cnt_indicator += 1

    def HT_DCPHASE(self):
        indicator = talib.HT_DCPHASE(self.close)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("HT_DCPHASE")
        self.total_cnt_indicator += 1

    def HT_PHASOR_inphase(self):
        HT_PHASOR_inphase,HT_PHASOR_quadrature = talib.HT_PHASOR(self.close)
        indicator = np.reshape(HT_PHASOR_inphase, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("HT_PHASOR_inphase")
        self.total_cnt_indicator += 1

    def HT_PHASOR_quadrature(self):
        HT_PHASOR_inphase,HT_PHASOR_quadrature = talib.HT_PHASOR(self.close)
        indicator = np.reshape(HT_PHASOR_quadrature, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("HT_PHASOR_quadrature")
        self.total_cnt_indicator += 1 

    def HT_SINE_sine(self):
        HT_SINE_sine,HT_SINE_leadsine  = talib.HT_SINE(self.close)
        indicator = np.reshape(HT_SINE_sine, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("HT_SINE_sine")
        self.total_cnt_indicator += 1 

    def HT_SINE_leadsine(self):
        HT_SINE_sine,HT_SINE_leadsine  = talib.HT_SINE(self.close)
        indicator = np.reshape(HT_SINE_leadsine, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("HT_SINE_leadsine")
        self.total_cnt_indicator += 1 

    def HT_TRENDMODE(self):
        indicator  = talib.HT_TRENDMODE(self.close)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("HT_TRENDMODE")
        self.total_cnt_indicator += 1 

    #動量指標
    # 1. ADX：平均趋向指数
    def ADX(self):
        indicator = talib.ADX(self.high, self.low, self.close, timeperiod=14)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("ADX")
        self.total_cnt_indicator += 1
    # 2. ADXR：平均趋向指数的趋向指数
    def ADXR(self):
        indicator = talib.ADXR(self.high, self.low, self.close, timeperiod=14)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("ADXR")
        self.total_cnt_indicator += 1
    # 3. APO ：价格震荡指数
    def APO(self):
        indicator = talib.APO(self.close, fastperiod=12, slowperiod=26, matype=0)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("APO")
        self.total_cnt_indicator += 1
    # 4. AROON ：阿隆指标
    def AROON_aroondown(self):
        AROON_aroondown, AROON_aroonup = talib.AROON(self.high, self.low, timeperiod=14)
        indicator = np.reshape(AROON_aroondown, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("AROON_aroondown")
        self.total_cnt_indicator += 1

    def AROON_aroonup(self):
        AROON_aroondown, AROON_aroonup = talib.AROON(self.high, self.low, timeperiod=14)
        indicator = np.reshape(AROON_aroonup, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("AROON_aroonup")
        self.total_cnt_indicator += 1  
   # 5.AROONOSC ：阿隆振荡
    def AROONOSC(self):
        indicator = talib.AROONOSC(self.high, self.low, timeperiod=14)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("AROONOSC")
        self.total_cnt_indicator += 1  
    # 6. BOP ：均势指标
    def BOP(self):
        indicator = talib.BOP(self.open, self.high, self.low, self.close)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("BOP")
        self.total_cnt_indicator += 1
    # 7. CCI ：顺势指标
    def CCI(self):
        indicator = talib.CCI(self.high, self.low, self.close, timeperiod=14)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CCI")
        self.total_cnt_indicator += 1
    # 8. CMO ：钱德动量摆动指标
    def CMO(self):
        indicator = talib.CMO(self.close, timeperiod=14)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CMO")
        self.total_cnt_indicator += 1
    # 9. DX ：动向指标或趋向指标
    def DX(self):
        indicator = talib.DX(self.high, self.low, self.close, timeperiod=14)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("DX")
        self.total_cnt_indicator += 1
    # 10. MACD:平滑异同移动平均线
    def MACD_macd(self):
        macd, macdsignal, macdhist = talib.MACD(self.close, fastperiod=12, slowperiod=26, signalperiod=9)
        indicator = np.reshape(macd, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("MACD_macd")
        self.total_cnt_indicator += 1
    
    def MACD_macdsignal(self):
        macd, macdsignal, macdhist = talib.MACD(self.close, fastperiod=12, slowperiod=26, signalperiod=9)
        indicator = np.reshape(macdsignal, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("MACD_macdsignal")
        self.total_cnt_indicator += 1
    
    def MACD_macdhist(self):
        macd, macdsignal, macdhist = talib.MACD(self.close, fastperiod=12, slowperiod=26, signalperiod=9)
        indicator = np.reshape(macdhist, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("MACD_macdhist")
        self.total_cnt_indicator += 1
    # 11. MACDEXT :MACD延伸
    def MACDEXT(self):
        ta_tuple = talib.MACDEXT(self.close, fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)
        for n, indicator in threading.enumerate(ta_tuple):
            indicator = np.reshape(indicator, (-1,1))
            self.technical_index = np.append(self.technical_index, indicator, axis = 1)
            self.technical_header.append(f"MACDEXT_{n}")
            self.total_cnt_indicator += 1
    # 12. MFI ：资金流量指标
    def MFI(self):
        indicator = talib.MFI(self.high, self.low, self.close, self.volume, timeperiod=14)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("MFI")
        self.total_cnt_indicator += 1
    # 13. MINUS_DI：DMI 中的DI指标 负方向指标
    def MINUS_DI(self):
        indicator = talib.MINUS_DI(self.high, self.low, self.close, timeperiod=14)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("MINUS_DI")
        self.total_cnt_indicator += 1
    # 14. MINUS_DM：上升动向值
    def MINUS_DM(self):
        indicator = talib.MINUS_DM(self.high, self.low, timeperiod=14)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("MINUS_DI")
        self.total_cnt_indicator += 1

    #重疊指標
    # 1.简单移动平均指标SMA
    def SMA(self):
        indicator = talib.SMA(self.close,5)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("SMA")
        self.total_cnt_indicator += 1 
    # 2.布林线BBANDS
    def upper(self):
        upper, middle, lower = talib.BBANDS(self.close,5,matype = talib.MA_Type.EMA)
        indicator = np.reshape(upper, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("upper")
        self.total_cnt_indicator += 1
    def middle(self):
        upper, middle, lower = talib.BBANDS(self.close,5,matype = talib.MA_Type.EMA)
        indicator = np.reshape(middle, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("middle")
        self.total_cnt_indicator += 1 
    def lower(self):
        upper, middle, lower = talib.BBANDS(self.close,5,matype = talib.MA_Type.EMA)
        indicator = np.reshape(lower, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("lower")
        self.total_cnt_indicator += 1 
    # 3. DEMA 双移动平均线:DEMA = 2*EMA-EMA(EMA)
    def DEMA(self):
        indicator = talib.DEMA(self.close, timeperiod = 30)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("DEMA")
        self.total_cnt_indicator += 1
    # 4. MA
    def MA(self):
        indicator = talib.MA(self.close, timeperiod = 10, matype = 0)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("MA")
        self.total_cnt_indicator += 1
    # 5. EMA
    def EMA(self):
        indicator = talib.EMA(self.close, timeperiod = 6)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("EMA")
        self.total_cnt_indicator += 1
    # 6.KAMA：考夫曼的自适应移动平均线
    def KAMA(self):
        indicator = talib.KAMA(self.close, timeperiod = 30)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("KAMA")
        self.total_cnt_indicator += 1
    # 7. MIDPRICE：阶段中点价格
    def MIDPOINT(self):
        indicator = talib.MIDPOINT(self.close, timeperiod = 14)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("MIDPOINT")
        self.total_cnt_indicator += 1
    # 8.SAR：抛物线指标
    def SAR(self):
        indicator = talib.SAR(self.high, self.low, acceleration=0, maximum=0)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("SAR")
        self.total_cnt_indicator += 1
    # 9.MIDPRICE：阶段中点价格（Midpoint Price over period)
    def MIDPRICE(self):
        indicator = talib.MIDPRICE(self.high, self.low, timeperiod=14)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("MIDPRICE")
        self.total_cnt_indicator += 1
    # 10. T3:三重移动平均线
    def T3(self):
        indicator = talib.T3(self.close, timeperiod = 5, vfactor = 0)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("T3")
        self.total_cnt_indicator += 1
    # # 11.TEMA：三重指数移动平均线
    def TEMA(self):
        indicator = talib.TEMA(self.close, timeperiod=30)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("TEMA")
        self.total_cnt_indicator += 1
    # 12.SAREXT:SAR的抛物面扩展
    def SAREXT(self):
        indicator = talib.SAREXT(self.high, self.low, startvalue=0, offsetonreverse=0, accelerationinitlong=0, accelerationlong=0, accelerationmaxlong=0, accelerationinitshort=0, accelerationshort=0, accelerationmaxshort=0)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("SAREXT")
        self.total_cnt_indicator += 1
    # 13.WMA：移动加权平均法
    def WMA(self):
        indicator = talib.WMA(self.close, timeperiod = 30)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("WMA")
        self.total_cnt_indicator += 1
    #
    def CDL2CROWS(self):
        indicator = talib.CDL2CROWS(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDL2CROWS")
        self.total_cnt_indicator += 1

    def CDL3BLACKCROWS(self):
        indicator = talib.CDL3BLACKCROWS(self.open, self.high, self.low, self.close)
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDL3BLACKCROWS")
        self.total_cnt_indicator += 1

    def CDL3INSIDE(self):
        indicator = talib.CDL3INSIDE(self.open, self.high, self.low, self.close)  
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDL3INSIDE")
        self.total_cnt_indicator += 1

    def CDL3LINESTRIKE(self):
        indicator = talib.CDL3LINESTRIKE(self.open, self.high, self.low, self.close)  
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDL3LINESTRIKE")
        self.total_cnt_indicator += 1

    def CDL3OUTSIDE(self):
        indicator = talib.CDL3OUTSIDE(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDL3OUTSIDE")
        self.total_cnt_indicator += 1

    def CDL3STARSINSOUTH(self):
        indicator = talib.CDL3STARSINSOUTH(self.open, self.high, self.low, self.close)  
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDL3STARSINSOUTH")
        self.total_cnt_indicator += 1

    def CDL3WHITESOLDIERS(self):
        indicator = talib.CDL3WHITESOLDIERS(self.open, self.high, self.low, self.close)  
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDL3WHITESOLDIERS")
        self.total_cnt_indicator += 1

    def CDLABANDONEDBABY(self):
        indicator = talib.CDLABANDONEDBABY(self.open, self.high, self.low, self.close, penetration=0) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLABANDONEDBABY")
        self.total_cnt_indicator += 1

    def CDLADVANCEBLOCK(self):
        indicator = talib.CDLADVANCEBLOCK(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLADVANCEBLOCK")
        self.total_cnt_indicator += 1

    def CDLBELTHOLD(self):
        indicator = talib.CDLBELTHOLD(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLBELTHOLD")
        self.total_cnt_indicator += 1

    def CDLBREAKAWAY(self):
        indicator = talib.CDLBREAKAWAY(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLBREAKAWAY")
        self.total_cnt_indicator += 1

    def CDLCLOSINGMARUBOZU(self):
        indicator = talib.CDLCLOSINGMARUBOZU(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLCLOSINGMARUBOZU")
        self.total_cnt_indicator += 1

    def CDLCONCEALBABYSWALL(self):
        indicator = talib.CDLCONCEALBABYSWALL(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLCONCEALBABYSWALL")
        self.total_cnt_indicator += 1

    def CDLCOUNTERATTACK(self):
        indicator = talib.CDLCOUNTERATTACK(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLCOUNTERATTACK")
        self.total_cnt_indicator += 1
 
    def CDLDARKCLOUDCOVER(self):
        indicator = talib.CDLDARKCLOUDCOVER(self.open, self.high, self.low, self.close, penetration=0) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLDARKCLOUDCOVER")
        self.total_cnt_indicator += 1

    def CDLDOJI(self):
        indicator = talib.CDLDOJI(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLDOJI")
        self.total_cnt_indicator += 1

    def CDLDOJISTAR(self):
        indicator = talib.CDLDOJISTAR(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLDOJISTAR")
        self.total_cnt_indicator += 1

    def CDLDRAGONFLYDOJI(self):
        indicator = talib.CDLDRAGONFLYDOJI(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLDRAGONFLYDOJI")
        self.total_cnt_indicator += 1

    def CDLENGULFING(self):
        indicator = talib.CDLENGULFING(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLENGULFING")
        self.total_cnt_indicator += 1
 
    def CDLEVENINGDOJISTAR(self):
        indicator = talib.CDLEVENINGDOJISTAR(self.open, self.high, self.low, self.close, penetration=0) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLEVENINGDOJISTAR")
        self.total_cnt_indicator += 1

    def CDLEVENINGSTAR(self):
        indicator = talib.CDLEVENINGSTAR(self.open, self.high, self.low, self.close, penetration=0) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLEVENINGSTAR")
        self.total_cnt_indicator += 1

    def CDLGAPSIDESIDEWHITE(self):
        indicator = talib.CDLGAPSIDESIDEWHITE(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLGAPSIDESIDEWHITE")
        self.total_cnt_indicator += 1

    def CDLGRAVESTONEDOJI(self):
        indicator = talib.CDLGRAVESTONEDOJI(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLGRAVESTONEDOJI")
        self.total_cnt_indicator += 1

    def CDLHAMMER(self):
        indicator = talib.CDLHAMMER(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLHAMMER")
        self.total_cnt_indicator += 1

    def CDLHANGINGMAN(self):
        indicator = talib.CDLHANGINGMAN(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLHANGINGMAN")
        self.total_cnt_indicator += 1

    def CDLHARAMI(self):
        indicator = talib.CDLHARAMI(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLHARAMI")
        self.total_cnt_indicator += 1

    def CDLHARAMICROSS(self):
        indicator = talib.CDLHARAMICROSS(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLHARAMICROSS")
        self.total_cnt_indicator += 1

    def CDLHIGHWAVE(self):
        indicator = talib.CDLHIGHWAVE(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLHIGHWAVE")
        self.total_cnt_indicator += 1

    def CDLHIKKAKE(self):
        indicator = talib.CDLHIKKAKE(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLHIKKAKE")
        self.total_cnt_indicator += 1

    def CDLHIKKAKEMOD(self):
        indicator = talib.CDLHIKKAKEMOD(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLHIKKAKEMOD")
        self.total_cnt_indicator += 1

    def CDLHOMINGPIGEON(self):
        indicator = talib.CDLHOMINGPIGEON(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLHOMINGPIGEON")
        self.total_cnt_indicator += 1
 
    def CDLIDENTICAL3CROWS(self):
        indicator = talib.CDLIDENTICAL3CROWS(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLIDENTICAL3CROWS")
        self.total_cnt_indicator += 1

    def CDLINNECK(self):
        indicator = talib.CDLINNECK(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLINNECK")
        self.total_cnt_indicator += 1

    def CDLINVERTEDHAMMER(self):
        indicator = talib.CDLINVERTEDHAMMER(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLINVERTEDHAMMER")
        self.total_cnt_indicator += 1

    def CDLKICKING(self):
        indicator = talib.CDLKICKING(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLKICKING")
        self.total_cnt_indicator += 1

    def CDLKICKINGBYLENGTH(self):
        indicator = talib.CDLKICKINGBYLENGTH(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLKICKINGBYLENGTH")
        self.total_cnt_indicator += 1

    def CDLLADDERBOTTOM(self):
        indicator = talib.CDLLADDERBOTTOM(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLLADDERBOTTOM")
        self.total_cnt_indicator += 1

    def CDLLONGLEGGEDDOJI(self):
        indicator = talib.CDLLONGLEGGEDDOJI(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLLONGLEGGEDDOJI")
        self.total_cnt_indicator += 1

    def CDLLONGLINE(self):
        indicator = talib.CDLLONGLINE(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLLONGLINE")
        self.total_cnt_indicator += 1

    def CDLMARUBOZU(self):
        indicator = talib.CDLMARUBOZU(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLMARUBOZU")
        self.total_cnt_indicator += 1

    def CDLMATCHINGLOW(self):
        indicator = talib.CDLMATCHINGLOW(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLMATCHINGLOW")
        self.total_cnt_indicator += 1

    def CDLMATHOLD(self):
        indicator = talib.CDLMATHOLD(self.open, self.high, self.low, self.close, penetration=0) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLMATHOLD")
        self.total_cnt_indicator += 1

    def CDLMORNINGDOJISTAR(self):
        indicator = talib.CDLMORNINGDOJISTAR(self.open, self.high, self.low, self.close, penetration=0) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLMORNINGDOJISTAR")
        self.total_cnt_indicator += 1

    def CDLMORNINGSTAR(self):
        indicator = talib.CDLMORNINGSTAR(self.open, self.high, self.low, self.close, penetration=0) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLMORNINGSTAR")
        self.total_cnt_indicator += 1

    def CDLONNECK(self):
        indicator = talib.CDLONNECK(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLONNECK")
        self.total_cnt_indicator += 1

    def CDLPIERCING(self):
        indicator = talib.CDLPIERCING(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLPIERCING")
        self.total_cnt_indicator += 1

    def CDLRICKSHAWMAN(self):
        indicator = talib.CDLRICKSHAWMAN(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLRICKSHAWMAN")
        self.total_cnt_indicator += 1

    def CDLRISEFALL3METHODS(self):
        indicator = talib.CDLRISEFALL3METHODS(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLRISEFALL3METHODS")
        self.total_cnt_indicator += 1

    def CDLSEPARATINGLINES(self):
        indicator = talib.CDLSEPARATINGLINES(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLSEPARATINGLINES")
        self.total_cnt_indicator += 1

    def CDLSHOOTINGSTAR(self):
        indicator = talib.CDLSHOOTINGSTAR(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLSHOOTINGSTAR")
        self.total_cnt_indicator += 1

    def CDLSHORTLINE(self):
        indicator = talib.CDLSHORTLINE(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLSHORTLINE")
        self.total_cnt_indicator += 1

    def CDLSPINNINGTOP(self):
        indicator = talib.CDLSPINNINGTOP(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLSPINNINGTOP")
        self.total_cnt_indicator += 1

    def CDLSTALLEDPATTERN(self):
        indicator = talib.CDLSTALLEDPATTERN(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLSTALLEDPATTERN")
        self.total_cnt_indicator += 1

    def CDLSTICKSANDWICH(self):
        indicator = talib.CDLSTICKSANDWICH(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLSTICKSANDWICH")
        self.total_cnt_indicator += 1

    def CDLTAKURI(self):
        indicator = talib.CDLTAKURI(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLTAKURI")
        self.total_cnt_indicator += 1

    def CDLTASUKIGAP(self):
        indicator = talib.CDLTASUKIGAP(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLTASUKIGAP")
        self.total_cnt_indicator += 1

    def CDLTHRUSTING(self):
        indicator = talib.CDLTHRUSTING(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLTHRUSTING")
        self.total_cnt_indicator += 1
 
    def CDLTRISTAR(self):
        indicator = talib.CDLTRISTAR(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLTRISTAR")
        self.total_cnt_indicator += 1

    def CDLUNIQUE3RIVER(self):
        indicator = talib.CDLUNIQUE3RIVER(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLUNIQUE3RIVER")
        self.total_cnt_indicator += 1

    def CDLUPSIDEGAP2CROWS(self):
        indicator = talib.CDLUPSIDEGAP2CROWS(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLUPSIDEGAP2CROWS")
        self.total_cnt_indicator += 1

    def CDLXSIDEGAP3METHODS(self):
        indicator = talib.CDLXSIDEGAP3METHODS(self.open, self.high, self.low, self.close) 
        indicator = np.reshape(indicator, (-1,1))
        self.technical_index = np.append(self.technical_index, indicator, axis = 1)
        self.technical_header.append("CDLXSIDEGAP3METHODS")
        self.total_cnt_indicator += 1

    def config_reading(self):
        for calling_indicator in  self.lib:
            if(calling_indicator == "HT_DCPERIOD"): self.HT_DCPERIOD()
            if(calling_indicator == "HT_DCPHASE"): self.HT_DCPHASE()
            if(calling_indicator == "HT_PHASOR_inphase"): self.HT_PHASOR_inphase()
            if(calling_indicator == "HT_PHASOR_quadrature"): self.HT_PHASOR_quadrature()
            if(calling_indicator == "HT_SINE_sine"): self.HT_SINE_sine()
            if(calling_indicator == "HT_SINE_leadsine"): self.HT_SINE_leadsine()
            if(calling_indicator == "HT_TRENDMODE"): self.HT_TRENDMODE()
            if(calling_indicator == "ADX"): self.ADX()
            if(calling_indicator == "ADXR"): self.ADXR()
            if(calling_indicator == "APO"): self.APO()
            if(calling_indicator == "AROON_aroondown"): self.AROON_aroondown()
            if(calling_indicator == "AROON_aroonup"): self.AROON_aroonup()
            if(calling_indicator == "AROONOSC"): self.AROONOSC()
            if(calling_indicator == "BOP"): self.BOP()
            if(calling_indicator == "CCI"): self.CCI()
            if(calling_indicator == "CMO"): self.CMO()
            if(calling_indicator == "DX"): self.DX()
            if(calling_indicator == "MACD_macd"): self.MACD_macd()
            if(calling_indicator == "MACD_macdsignal"): self.MACD_macdsignal()
            if(calling_indicator == "MACD_macdhist"): self.MACD_macdhist()
            if(calling_indicator == "MFI"): self.MFI()
            if(calling_indicator == "MINUS_DI"): self.MINUS_DI()
            if(calling_indicator == "MINUS_DM"): self.MINUS_DM()
            if(calling_indicator == "SMA"): self.SMA()
            if(calling_indicator == "upper"): self.upper()
            if(calling_indicator == "middle"): self.middle()
            if(calling_indicator == "lower"): self.lower()
            if(calling_indicator == "DEMA"): self.DEMA()
            if(calling_indicator == "EMA"): self.EMA()
            if(calling_indicator == "KAMA"): self.KAMA()
            if(calling_indicator == "MA"): self.MA()
            if(calling_indicator == "SAR"): self.SAR()
            if(calling_indicator == "SAREXT"): self.SAREXT()
            if(calling_indicator == "T3"): self.T3()
            if(calling_indicator == "TEMA"): self.TEMA()
            if(calling_indicator == "WMA"): self.WMA()
            if(calling_indicator == "CDL2CROWS"): self.CDL2CROWS()
            if(calling_indicator == "CDL3BLACKCROWS"): self.CDL3BLACKCROWS()
            if(calling_indicator == "CDL3INSIDE"): self.CDL3INSIDE()
            if(calling_indicator == "CDL3LINESTRIKE"): self.CDL3LINESTRIKE()
            if(calling_indicator == "CDL3OUTSIDE"): self.CDL3OUTSIDE()
            if(calling_indicator == "CDL3STARSINSOUTH"): self.CDL3STARSINSOUTH()
            if(calling_indicator == "CDL3WHITESOLDIERS"): self.CDL3WHITESOLDIERS()
            if(calling_indicator == "CDLABANDONEDBABY"): self.CDLABANDONEDBABY()
            if(calling_indicator == "CDLADVANCEBLOCK"): self.CDLADVANCEBLOCK()
            if(calling_indicator == "CDLBELTHOLD"): self.CDLBELTHOLD()
            if(calling_indicator == "CDLBREAKAWAY"): self.CDLBREAKAWAY()
            if(calling_indicator == "CDLCLOSINGMARUBOZU"): self.CDLCLOSINGMARUBOZU()
            if(calling_indicator == "CDLCONCEALBABYSWALL"): self.CDLCONCEALBABYSWALL()
            if(calling_indicator == "CDLCOUNTERATTACK"): self.CDLCOUNTERATTACK()
            if(calling_indicator == "CDLDARKCLOUDCOVER"): self.CDLDARKCLOUDCOVER()
            if(calling_indicator == "CDLDOJI"): self.CDLDOJI()
            if(calling_indicator == "CDLDOJISTAR"): self.CDLDOJISTAR()
            if(calling_indicator == "CDLDRAGONFLYDOJI"): self.CDLDRAGONFLYDOJI()
            if(calling_indicator == "CDLENGULFING"): self.CDLENGULFING()
            if(calling_indicator == "CDLEVENINGDOJISTAR"): self.CDLEVENINGDOJISTAR()
            if(calling_indicator == "CDLEVENINGSTAR"): self.CDLEVENINGSTAR()
            if(calling_indicator == "CDLGAPSIDESIDEWHITE"): self.CDLGAPSIDESIDEWHITE()
            if(calling_indicator == "CDLGRAVESTONEDOJI"): self.CDLGRAVESTONEDOJI()
            if(calling_indicator == "CDLHAMMER"): self.CDLHAMMER()
            if(calling_indicator == "CDLHANGINGMAN"): self.CDLHANGINGMAN()
            if(calling_indicator == "CDLHARAMI"): self.CDLHARAMI()
            if(calling_indicator == "CDLHARAMICROSS"): self.CDLHARAMICROSS()
            if(calling_indicator == "CDLHIGHWAVE"): self.CDLHIGHWAVE()
            if(calling_indicator == "CDLHIKKAKE"): self.CDLHIKKAKE()
            if(calling_indicator == "CDLHIKKAKEMOD"): self.CDLHIKKAKEMOD()
            if(calling_indicator == "CDLHOMINGPIGEON"): self.CDLHOMINGPIGEON()
            if(calling_indicator == "CDLIDENTICAL3CROWS"): self.CDLIDENTICAL3CROWS()
            if(calling_indicator == "CDLINNECK"): self.CDLINNECK()
            if(calling_indicator == "CDLINVERTEDHAMMER"): self.CDLINVERTEDHAMMER()
            if(calling_indicator == "CDLKICKING"): self.CDLKICKING()
            if(calling_indicator == "CDLKICKINGBYLENGTH"): self.CDLKICKINGBYLENGTH()
            if(calling_indicator == "CDLLADDERBOTTOM"): self.CDLLADDERBOTTOM()
            if(calling_indicator == "CDLLONGLEGGEDDOJI"): self.CDLLONGLEGGEDDOJI()
            if(calling_indicator == "CDLLONGLINE"): self.CDLLONGLINE()
            if(calling_indicator == "CDLMARUBOZU"): self.CDLMARUBOZU()
            if(calling_indicator == "CDLMATCHINGLOW"): self.CDLMATCHINGLOW()
            if(calling_indicator == "CDLMATHOLD"): self.CDLMATHOLD()
            if(calling_indicator == "CDLMORNINGDOJISTAR"): self.CDLMORNINGDOJISTAR()
            if(calling_indicator == "CDLMORNINGSTAR"): self.CDLMORNINGSTAR()
            if(calling_indicator == "CDLONNECK"): self.CDLONNECK()
            if(calling_indicator == "CDLPIERCING"): self.CDLPIERCING()
            if(calling_indicator == "CDLRICKSHAWMAN"): self.CDLRICKSHAWMAN()
            if(calling_indicator == "CDLRISEFALL3METHODS"): self.CDLRISEFALL3METHODS()
            if(calling_indicator == "CDLSEPARATINGLINES"): self.CDLSEPARATINGLINES()
            if(calling_indicator == "CDLSHOOTINGSTAR"): self.CDLSHOOTINGSTAR()
            if(calling_indicator == "CDLSHORTLINE"): self.CDLSHORTLINE()
            if(calling_indicator == "CDLSPINNINGTOP"): self.CDLSPINNINGTOP()
            if(calling_indicator == "CDLSTALLEDPATTERN"): self.CDLSTALLEDPATTERN()
            if(calling_indicator == "CDLSTICKSANDWICH"): self.CDLSTICKSANDWICH()
            if(calling_indicator == "CDLTAKURI"): self.CDLTAKURI()
            if(calling_indicator == "CDLTASUKIGAP"): self.CDLTASUKIGAP()
            if(calling_indicator == "CDLTHRUSTING"): self.CDLTHRUSTING()
            if(calling_indicator == "CDLTRISTAR"): self.CDLTRISTAR()
            if(calling_indicator == "CDLUNIQUE3RIVER"): self.CDLUNIQUE3RIVER()
            if(calling_indicator == "CDLUPSIDEGAP2CROWS"): self.CDLUPSIDEGAP2CROWS()
            if(calling_indicator == "CDLXSIDEGAP3METHODS"): self.CDLXSIDEGAP3METHODS()

    def feature_process(self,df,headers) :
        all_data_tmp=[]
        cnt_row = 0
        cnt_col = 0
        all_head=[]

        try :
            self.technical_index=[]
            self.technical_header=[]
            self.total_cnt_indicator=0
            ##補0
            divider = 0.0
            for i in range(df.shape[0]):
                if (i==0):
                    if (df[i, 1]!=0):
                        divider=df[i, 1]
                    else:
                        raise ZeroDivisionError
                    continue
                if (df[i, 1] == 0): ##如果開盤價是0
                    df[i, 1] = df[i-1, 4]
                    df[i, 2] = df[i-1, 4]
                    df[i, 3] = df[i-1, 4]
                    df[i, 4] = df[i-1, 4]
                elif (divider==0):
                    divider=df[i, 1]
                    
            self.open  = df[:,1]/divider
            self.high  = df[:,2]/divider
            self.low   = df[:,3]/divider
            self.close = df[:,4]/divider
            self.volume = df[:,5]
            self.technical_index = df
            self.config_reading()
            df = self.technical_index
            all_data_tmp = np.append(all_data_tmp, df)
            cnt_row += df.shape[0]
            cnt_col = df.shape[1]
        except IndexError :
            print('資料有缺失在{}\n'.format(file))
        all_head = headers + self.technical_header

        all_data= np.reshape(all_data_tmp, (cnt_row, cnt_col)) ##重新reshape
        return all_data, all_head, self.total_cnt_indicator, cnt_col

## Training

In [ ]:
import numpy as np
from scipy.stats import kurtosis
from scipy.stats import skew
from matplotlib import pyplot as plt
from matplotlib.pyplot import MultipleLocator
import json
from reg_model import regression_model
import os
class output_reg(regression_model):
    def create_folder(self):
        directory = "output"
        parent_dir = self.directory
        path_graph = os.path.join(parent_dir, directory)
        os.mkdir(path_graph)
        return path_graph

    def statistics(self, data):
        mean = np.mean(data)
        std = np.std(data)
        skewness = skew (data)
        kurt = kurtosis(data)
        return mean, std, skewness, kurt

    def feature_details(self, path):
        #create feature_details folder
        directory = "feature_details"
        parent_dir = path
        path = os.path.join(parent_dir, directory)
        os.mkdir(path)

        X = self.x
        Y = self.y
        ##label detail
        label = np.reshape(Y, (-1, 1))
        mean = np.mean(label)
        std = np.std(label)
        skewness = skew(label)
        kurt = kurtosis(label)
        skewness = np.float64(skewness)
        kurt = np.float64(kurt)
        plt.hist(x=label, bins=20)
        plt.title("Mean : {:.3f}, Std: {:.3f}, Skewness : {:.3f}, Kurtosis: {:.3f}".format(mean, std, skewness, kurt))
        plt.savefig("{}/label.png".format(path))
        self.plot_clear()
        
        ##feature detail
        feature_result =[]
        for feature in range(X.shape[1]):
            mean = np.mean(X[:, feature])
            std = np.std(X[:, feature])
            skewness = skew (X[:, feature])
            kurt = kurtosis(X[:, feature])
            feature_result_tmp =[mean, std, skewness, kurt]
            feature_result = np.append(feature_result, feature_result_tmp)
        feature_result= np.reshape(feature_result, (X.shape[1], 4)) ##重新reshape

        for feature in range(X.shape[1]):
            mean = feature_result[feature,0]
            std = feature_result[feature,1]
            skewness = feature_result[feature,2]
            kurt = feature_result[feature,3]
            
            x_distribution = (X[:,feature]- np.mean(X[:,feature]))/np.std(X[:,feature])
            plt.hist(x_distribution,range=[-1, 1], bins=20) ##設定xmin 和 xmax才會畫得出來
            plt.title("Mean : {:.3f}, Std: {:.3f}, Skewness : {:.3f}, Kurtosis: {:.3f}".format(mean, std, skewness, kurt))
            plt.savefig("{}/".format(path)+self.headers[feature]+".png")
            self.plot_clear()

    def feature_label_time_series(self, path):
        #create feature_label_time_series folder
        directory = "time_series"
        parent_dir = path
        path_graph = os.path.join(parent_dir, directory)
        os.mkdir(path_graph)

        X = self.x
        Y = self.y
        ##label time series
        self.plot_clear()
        plt.plot(np.arange(0, X.shape[0], 1), Y)
        plt.title("label")
        plt.savefig("{}/label.png".format(path_graph))
        self.plot_clear()
        for feature in range(X.shape[1]):
            self.plot_clear()
            plt.plot(np.arange(0, X.shape[0], 1), X[:, feature])
            plt.title("{}".format(self.headers[feature+1]))
            plt.savefig("{}/".format(path_graph)+self.headers[feature+1]+".png")
            self.plot_clear()

    def output_graph(self, path):
        #create graph and result folder
        directory_graph = "graph"
        directory_result = "result"
        parent_dir = path
        path_graph = os.path.join(parent_dir, directory_graph)
        path_result = os.path.join(parent_dir, directory_result)
        os.mkdir(path_graph)
        os.mkdir(path_result)
        with open("{}/output_reg.json".format(path_result), "w") as outfile:
            json.dump(self.dict, outfile, indent = 4)
        ##index
        train_index = range(0, self.train_y.shape[0])
        test_index = range(0, self.test_y.shape[0])

        ##linear regression 
        plt.scatter(self.train_y, self.y_pred_train[:,0], c="red", s=10)
        plt.plot([-0.02, 0.02], [-0.02, 0.02], '--', color = 'black', linewidth = 2)
        plt.savefig("{}/linear_regression_insample.png".format(path_graph))
        self.plot_clear()
        plt.scatter(self.test_y, self.y_pred[:, 0], c="red", s=10)
        plt.plot([-0.02, 0.02], [-0.02, 0.02], '--', color = 'black', linewidth = 2)
        plt.savefig("{}/linear_regression_outsample.png".format(path_graph))
        self.plot_clear()

        ## detail
        mean, std, skewness, kurt = self.statistics(self.train_y)
        self.insample_y_detail["Name"].append("linear_regression")
        self.insample_y_detail["Mean"].append(mean)
        self.insample_y_detail["Std"].append(std)
        self.insample_y_detail["Skewness"].append(skewness)
        self.insample_y_detail["Kurtosis"].append(kurt)
        mean, std, skewness, kurt = self.statistics(self.y_pred_train[:,0])
        self.insample_predict_detail["Name"].append("linear_regression")
        self.insample_predict_detail["Mean"].append(mean)
        self.insample_predict_detail["Std"].append(std)
        self.insample_predict_detail["Skewness"].append(skewness)
        self.insample_predict_detail["Kurtosis"].append(kurt)
        mean, std, skewness, kurt = self.statistics(self.test_y)
        self.outsample_y_detail["Name"].append("linear_regression")
        self.outsample_y_detail["Mean"].append(mean)
        self.outsample_y_detail["Std"].append(std)
        self.outsample_y_detail["Skewness"].append(skewness)
        self.outsample_y_detail["Kurtosis"].append(kurt)
        mean, std, skewness, kurt = self.statistics(self.y_pred[:,0])
        self.outsample_predict_detail["Name"].append("linear_regression")
        self.outsample_predict_detail["Mean"].append(mean)
        self.outsample_predict_detail["Std"].append(std)
        self.outsample_predict_detail["Skewness"].append(skewness)
        self.outsample_predict_detail["Kurtosis"].append(kurt)

        ##linear regression predict distribution
        insample_y_distribution = (self.train_y- np.mean(self.train_y))/np.std(self.train_y)
        insample_predict_distribution = (self.y_pred_train[:,0]- np.mean(self.y_pred_train[:,0]))/np.std(self.y_pred_train[:,0])
        plt.hist(insample_y_distribution, alpha=0.5, label='Pred', color = "skyblue", bins=20)
        plt.hist(insample_predict_distribution, alpha=0.5, label='True', color = "palegreen", bins=20)
        plt.legend(loc='upper right')
        plt.savefig("{}/linear_regression_insample_distribution.png".format(path_graph))
        self.plot_clear()
        outsample_y_distribution = (self.test_y- np.mean(self.test_y))/np.std(self.test_y)
        outsample_predict_distribution = (self.y_pred[:,0]- np.mean(self.y_pred[:,0]))/np.std(self.y_pred[:,0])
        plt.hist(outsample_y_distribution, alpha=0.5, label='Pred', color = "skyblue", bins=20)
        plt.hist(outsample_predict_distribution, alpha=0.5, label='True', color = "palegreen", bins=20)
        plt.legend(loc='upper right')
        plt.savefig("{}/linear_regression_outsample_distribution.png".format(path_graph))
        self.plot_clear()

        ##lasso
        plt.scatter(self.train_y, self.y_pred_train[:, 1], c="red", s=10)
        plt.plot([-0.02, 0.02], [-0.02, 0.02], '--', color = 'black', linewidth = 2)
        plt.savefig("{}/lasso_insample.png".format(path_graph))
        self.plot_clear()
        plt.scatter(self.test_y, self.y_pred[:, 1], c="red", s=10)
        plt.plot([-0.02, 0.02], [-0.02, 0.02], '--', color = 'black', linewidth = 2)
        plt.savefig("{}/lasso_outsample.png".format(path_graph))
        self.plot_clear()

        ## detail
        mean, std, skewness, kurt = self.statistics(self.train_y)
        self.insample_y_detail["Name"].append("lasso")
        self.insample_y_detail["Mean"].append(mean)
        self.insample_y_detail["Std"].append(std)
        self.insample_y_detail["Skewness"].append(skewness)
        self.insample_y_detail["Kurtosis"].append(kurt)
        mean, std, skewness, kurt = self.statistics(self.y_pred_train[:, 1])
        self.insample_predict_detail["Name"].append("lasso")
        self.insample_predict_detail["Mean"].append(mean)
        self.insample_predict_detail["Std"].append(std)
        self.insample_predict_detail["Skewness"].append(skewness)
        self.insample_predict_detail["Kurtosis"].append(kurt)
        mean, std, skewness, kurt = self.statistics(self.test_y)
        self.outsample_y_detail["Name"].append("lasso")
        self.outsample_y_detail["Mean"].append(mean)
        self.outsample_y_detail["Std"].append(std)
        self.outsample_y_detail["Skewness"].append(skewness)
        self.outsample_y_detail["Kurtosis"].append(kurt)
        mean, std, skewness, kurt = self.statistics(self.y_pred[:, 1])
        self.outsample_predict_detail["Name"].append("lasso")
        self.outsample_predict_detail["Mean"].append(mean)
        self.outsample_predict_detail["Std"].append(std)
        self.outsample_predict_detail["Skewness"].append(skewness)
        self.outsample_predict_detail["Kurtosis"].append(kurt)

        ##lasso predict distribution
        insample_y_distribution = (self.train_y- np.mean(self.train_y))/np.std(self.train_y)
        insample_predict_distribution = (self.y_pred_train[:,1]- np.mean(self.y_pred_train[:,1]))/np.std(self.y_pred_train[:,1])
        plt.hist(insample_y_distribution, alpha=0.5, label='Pred', color = "skyblue", bins=20)
        plt.hist(insample_predict_distribution, alpha=0.5, label='True', color = "palegreen", bins=20)
        plt.legend(loc='upper right')
        plt.savefig("{}/lasso_insample_distribution.png".format(path_graph))
        self.plot_clear()
        outsample_y_distribution = (self.test_y- np.mean(self.test_y))/np.std(self.test_y)
        outsample_predict_distribution = (self.y_pred[:,1]- np.mean(self.y_pred[:,1]))/np.std(self.y_pred[:,1])
        plt.hist(outsample_y_distribution, alpha=0.5, label='Pred', color = "skyblue", bins=20)
        plt.hist(outsample_predict_distribution, alpha=0.5, label='True', color = "palegreen", bins=20)
        plt.legend(loc='upper right')
        plt.savefig("{}/lasso_outsample_distribution.png".format(path_graph))
        self.plot_clear()

        ##ridge
        plt.scatter(self.train_y, self.y_pred_train[:, 2], c="red", s=10)
        plt.plot([-0.02, 0.02], [-0.02, 0.02], '--', color = 'black', linewidth = 2)
        plt.savefig("{}/ridge_insample.png".format(path_graph))
        self.plot_clear()
        plt.scatter(self.test_y, self.y_pred[:, 2], c="red", s=10)
        plt.plot([-0.02, 0.02], [-0.02, 0.02], '--', color = 'black', linewidth = 2)
        plt.savefig("{}/ridge_outsample.png".format(path_graph))
        self.plot_clear()

        ## detail
        mean, std, skewness, kurt = self.statistics(self.train_y)
        self.insample_y_detail["Name"].append("ridge")
        self.insample_y_detail["Mean"].append(mean)
        self.insample_y_detail["Std"].append(std)
        self.insample_y_detail["Skewness"].append(skewness)
        self.insample_y_detail["Kurtosis"].append(kurt)
        mean, std, skewness, kurt = self.statistics(self.y_pred_train[:, 2])
        self.insample_predict_detail["Name"].append("ridge")
        self.insample_predict_detail["Mean"].append(mean)
        self.insample_predict_detail["Std"].append(std)
        self.insample_predict_detail["Skewness"].append(skewness)
        self.insample_predict_detail["Kurtosis"].append(kurt)
        mean, std, skewness, kurt = self.statistics(self.test_y)
        self.outsample_y_detail["Name"].append("ridge")
        self.outsample_y_detail["Mean"].append(mean)
        self.outsample_y_detail["Std"].append(std)
        self.outsample_y_detail["Skewness"].append(skewness)
        self.outsample_y_detail["Kurtosis"].append(kurt)
        mean, std, skewness, kurt = self.statistics(self.y_pred[:, 2])
        self.outsample_predict_detail["Name"].append("ridge")
        self.outsample_predict_detail["Mean"].append(mean)
        self.outsample_predict_detail["Std"].append(std)
        self.outsample_predict_detail["Skewness"].append(skewness)
        self.outsample_predict_detail["Kurtosis"].append(kurt)

        ##ridge predict distribution
        insample_y_distribution = (self.train_y- np.mean(self.train_y))/np.std(self.train_y)
        insample_predict_distribution = (self.y_pred_train[:,2]- np.mean(self.y_pred_train[:,2]))/np.std(self.y_pred_train[:,2])
        plt.hist(insample_y_distribution, alpha=0.5, label='Pred', color = "skyblue", bins=20)
        plt.hist(insample_predict_distribution, alpha=0.5, label='True', color = "palegreen", bins=20)
        plt.legend(loc='upper right')
        plt.savefig("{}/ridge_insample_distribution.png".format(path_graph))
        self.plot_clear()
        outsample_y_distribution = (self.test_y- np.mean(self.test_y))/np.std(self.test_y)
        outsample_predict_distribution = (self.y_pred[:,2]- np.mean(self.y_pred[:,2]))/np.std(self.y_pred[:,2])
        plt.hist(outsample_y_distribution, alpha=0.5, label='Pred', color = "skyblue", bins=20)
        plt.hist(outsample_predict_distribution, alpha=0.5, label='True', color = "palegreen", bins=20)
        plt.legend(loc='upper right')
        plt.savefig("{}/ridge_outsample_distribution.png".format(path_graph))
        self.plot_clear()

        ##random_forest
        plt.scatter(self.train_y, self.y_pred_train[:, 3], c="red", s=10)
        plt.plot([-0.02, 0.02], [-0.02, 0.02], '--', color = 'black', linewidth = 2)
        plt.savefig("{}/random_forest_insample.png".format(path_graph))
        self.plot_clear()
        plt.scatter(self.test_y, self.y_pred[:, 3], c="red", s=10)
        plt.plot([-0.02, 0.02], [-0.02, 0.02], '--', color = 'black', linewidth = 2)
        plt.savefig("{}/random_forest_outsample.png".format(path_graph))
        self.plot_clear()

        ## detail
        mean, std, skewness, kurt = self.statistics(self.train_y)
        self.insample_y_detail["Name"].append("random_forest")
        self.insample_y_detail["Mean"].append(mean)
        self.insample_y_detail["Std"].append(std)
        self.insample_y_detail["Skewness"].append(skewness)
        self.insample_y_detail["Kurtosis"].append(kurt)
        mean, std, skewness, kurt = self.statistics(self.y_pred_train[:, 3])
        self.insample_predict_detail["Name"].append("random_forest")
        self.insample_predict_detail["Mean"].append(mean)
        self.insample_predict_detail["Std"].append(std)
        self.insample_predict_detail["Skewness"].append(skewness)
        self.insample_predict_detail["Kurtosis"].append(kurt)
        mean, std, skewness, kurt = self.statistics(self.test_y)
        self.outsample_y_detail["Name"].append("random_forest")
        self.outsample_y_detail["Mean"].append(mean)
        self.outsample_y_detail["Std"].append(std)
        self.outsample_y_detail["Skewness"].append(skewness)
        self.outsample_y_detail["Kurtosis"].append(kurt)
        mean, std, skewness, kurt = self.statistics(self.y_pred[:, 3])
        self.outsample_predict_detail["Name"].append("random_forest")
        self.outsample_predict_detail["Mean"].append(mean)
        self.outsample_predict_detail["Std"].append(std)
        self.outsample_predict_detail["Skewness"].append(skewness)
        self.outsample_predict_detail["Kurtosis"].append(kurt)

        ##ridge predict distribution
        insample_y_distribution = (self.train_y- np.mean(self.train_y))/np.std(self.train_y)
        insample_predict_distribution = (self.y_pred_train[:,3]- np.mean(self.y_pred_train[:,3]))/np.std(self.y_pred_train[:,3])
        plt.hist(insample_y_distribution, alpha=0.5, label='Pred', color = "skyblue", bins=20)
        plt.hist(insample_predict_distribution, alpha=0.5, label='True', color = "palegreen", bins=20)
        plt.legend(loc='upper right')
        plt.savefig("{}/random_forest_insample_distribution.png".format(path_graph))
        self.plot_clear()
        outsample_y_distribution = (self.test_y- np.mean(self.test_y))/np.std(self.test_y)
        outsample_predict_distribution = (self.y_pred[:,3]- np.mean(self.y_pred[:,3]))/np.std(self.y_pred[:,3])
        plt.hist(outsample_y_distribution, alpha=0.5, label='Pred', color = "skyblue", bins=20)
        plt.hist(outsample_predict_distribution, alpha=0.5, label='True', color = "palegreen", bins=20)
        plt.legend(loc='upper right')
        plt.savefig("{}/random_forest_outsample_distribution.png".format(path_graph))
        self.plot_clear()

        ##lgbm
        plt.scatter(self.train_y, self.y_pred_train[:, 4], c="red", s=10)
        plt.plot([-0.02, 0.02], [-0.02, 0.02], '--', color = 'black', linewidth = 2)
        plt.savefig("{}/lgbm_insample.png".format(path_graph))
        self.plot_clear()
        plt.scatter(self.test_y, self.y_pred[:, 4], c="red", s=10)
        plt.plot([-0.02, 0.02], [-0.02, 0.02], '--', color = 'black', linewidth = 2)
        plt.savefig("{}/lgbm_outsample.png".format(path_graph))
        self.plot_clear()

        ## detail
        mean, std, skewness, kurt = self.statistics(self.train_y)
        self.insample_y_detail["Name"].append("lgbm")
        self.insample_y_detail["Mean"].append(mean)
        self.insample_y_detail["Std"].append(std)
        self.insample_y_detail["Skewness"].append(skewness)
        self.insample_y_detail["Kurtosis"].append(kurt)
        mean, std, skewness, kurt = self.statistics(self.y_pred_train[:, 4])
        self.insample_predict_detail["Name"].append("lgbm")
        self.insample_predict_detail["Mean"].append(mean)
        self.insample_predict_detail["Std"].append(std)
        self.insample_predict_detail["Skewness"].append(skewness)
        self.insample_predict_detail["Kurtosis"].append(kurt)
        mean, std, skewness, kurt = self.statistics(self.test_y)
        self.outsample_y_detail["Name"].append("lgbm")
        self.outsample_y_detail["Mean"].append(mean)
        self.outsample_y_detail["Std"].append(std)
        self.outsample_y_detail["Skewness"].append(skewness)
        self.outsample_y_detail["Kurtosis"].append(kurt)
        mean, std, skewness, kurt = self.statistics(self.y_pred[:, 4])
        self.outsample_predict_detail["Name"].append("lgbm")
        self.outsample_predict_detail["Mean"].append(mean)
        self.outsample_predict_detail["Std"].append(std)
        self.outsample_predict_detail["Skewness"].append(skewness)
        self.outsample_predict_detail["Kurtosis"].append(kurt)

        ##lgbm predict distribution
        insample_y_distribution = (self.train_y- np.mean(self.train_y))/np.std(self.train_y)
        insample_predict_distribution = (self.y_pred_train[:,4]- np.mean(self.y_pred_train[:,4]))/np.std(self.y_pred_train[:,4])

        plt.hist(insample_y_distribution, alpha=0.5, label='Pred', color = "skyblue", bins=20)
        plt.hist(insample_predict_distribution, alpha=0.5, label='True', color = "palegreen", bins=20)
        plt.legend(loc='upper right')
        plt.savefig("{}/lgbm_insample_distribution.png".format(path_graph))
        self.plot_clear()
        outsample_y_distribution = (self.test_y- np.mean(self.test_y))/np.std(self.test_y)
        outsample_predict_distribution = (self.y_pred[:,4]- np.mean(self.y_pred[:,4]))/np.std(self.y_pred[:,4])
        plt.hist(outsample_y_distribution, alpha=0.5, label='Pred', color = "skyblue", bins=20)
        plt.hist(outsample_predict_distribution, alpha=0.5, label='True', color = "palegreen", bins=20)
        plt.legend(loc='upper right')
        plt.savefig("{}/lgbm_outsample_distribution.png".format(path_graph))
        self.plot_clear()

        with open("{}/insample_y_statistics.json".format(parent_dir), "w") as outfile:
            json.dump(self.insample_y_detail, outfile, indent = 4)
        with open("{}/insample_predict_statistics.json".format(parent_dir), "w") as outfile:
            json.dump(self.insample_predict_detail, outfile, indent = 4)
        with open("{}/outsample_y_statistics.json".format(parent_dir), "w") as outfile:
            json.dump(self.outsample_y_detail, outfile, indent = 4)
        with open("{}/outsample_predict_statistics.json".format(parent_dir), "w") as outfile:
            json.dump(self.outsample_predict_detail, outfile, indent = 4)

    def plot_clear(self):
        plt.figure().clear()
        plt.close()
        plt.cla()
        plt.clf()

    def show_result(self):
        super().train()
        path = self.create_folder()
        self.feature_details(path)
        self.feature_label_time_series(path)
        self.output_graph(path)

# Main

In [ ]:
import numpy as np
from datetime import datetime
import time
import os

In [ ]:
 lib = ["HT_DCPERIOD",	"HT_DCPHASE",	"HT_PHASOR_inphase",	"HT_PHASOR_quadrature",	"HT_SINE_sine",	
    "HT_SINE_leadsine",	"HT_TRENDMODE",	"ADX",	"ADXR",	"APO",	"AROON_aroondown",	"AROON_aroonup",	"AROONOSC",	
    "BOP",	"CCI",	"CMO",	"DX",	"MACD_macd",	"MACD_macdsignal",	"MACD_macdhist", 
    "MFI",	"MINUS_DI",	"MINUS_DM",	" SMA",	"upper",	"middle",	" lower",	"DEMA",	"EMA",	"KAMA",	"MA",	"MIDPOINT",	
    "SAR",	"SAREXT",	"T3",	"TEMA",	"WMA",	"CDL2CROWS",	"CDL3BLACKCROWS",	"CDL3INSIDE",	"CDL3LINESTRIKE",	" CDL3OUTSIDE",	
    "CDL3STARSINSOUTH",	"CDL3WHITESOLDIERS",	"CDLABANDONEDBABY",	"CDLADVANCEBLOCK",	"CDLBELTHOLD",	"CDLBREAKAWAY",	"CDLCLOSINGMARUBOZU",
    "CDLCONCEALBABYSWALL",	"CDLCOUNTERATTACK",	"CDLDARKCLOUDCOVER", "CDLDOJI",	"CDLDOJISTAR",	"CDLDRAGONFLYDOJI",	"CDLENGULFING",	"CDLEVENINGDOJISTAR",	
    "CDLEVENINGSTAR",	"CDLGAPSIDESIDEWHITE",	"CDLGRAVESTONEDOJI",	"CDLHAMMER",	"CDLHANGINGMAN",
        "CDLHARAMI",	"CDLHARAMICROSS",	"CDLHIGHWAVE",	"CDLHIKKAKE",	
    "CDLHIKKAKEMOD",	"CDLHOMINGPIGEON",	"CDLIDENTICAL3CROWS",	"CDLINNECK",	"CDLINVERTEDHAMMER",	"CDLKICKING",	"CDLKICKINGBYLENGTH",	"CDLLADDERBOTTOM",	"CDLLONGLEGGEDDOJI",	
    "CDLLONGLINE",	"CDLMARUBOZU",	"CDLMATCHINGLOW",	"CDLMATHOLD",	"CDLMORNINGDOJISTAR",	"CDLMORNINGSTAR",	"CDLONNECK",	"CDLPIERCING",	"CDLRICKSHAWMAN",	"CDLRISEFALL3METHODS",	
    "CDLSEPARATINGLINES",	"CDLSHOOTINGSTAR",	"CDLSHORTLINE",	"CDLSPINNINGTOP",	" CDLSTALLEDPATTERN",	"CDLSTICKSANDWICH",	"CDLTAKURI",	"CDLTASUKIGAP",	"CDLTHRUSTING",	"CDLTRISTAR",
    	"CDLUNIQUE3RIVER",	"CDLUPSIDEGAP2CROWS",	"CDLXSIDEGAP3METHODS"]

feature_gen = get_indicator(lib)
data, headers, total_indicator, total_row = feature_gen.feature_process(df_train_00.to_numpy(),df_train_00.columns.to_list())
print(data.shape)

In [ ]:
# for i in range(len(headers)):
#     print(headers[i],":",np.isnan(data[:,i]).sum())
# drop nan
data = data[~np.isnan(data).any(axis=1), :]
print(data.shape)
data = np.reshape(data, (-1, total_row))
print(data.shape)

In [ ]:

now = datetime.now()
current_time = str(now.month)+"_"+str(now.day)+"_"+now.strftime("%H_%M_%S")
parent_dir = "/kaggle/working/"
directory = current_time
path = os.path.join(parent_dir, directory)
os.mkdir(path)

In [ ]:
candle_tick = output_reg(data, headers, path)
candle_tick.show_result()